# kNN classifier code

In [ ]:
#Importing the required modules
import numpy as np
from scipy.stats import mode
 
#Euclidean Distance
def eucledian(p1,p2):
    dist = np.sqrt(np.sum((p1-p2)**2))
    return dist
 
#Function to calculate KNN
def predict(x_train, y , x_input, k):
    op_labels = []
    #Loop through the Datapoints to be classified
    for item in x_input: 
        #Array to store distances
        point_dist = []
        #Loop through each training Data
        for j in range(len(x_train)): 
            distances = eucledian(np.array(x_train[j,:]) , item)#distance.euclidean()# 
            #Calculating the distance
            point_dist.append(distances) 
        point_dist = np.array(point_dist) 
        #Sorting the array while preserving the index, and keeping the first K datapoints
        dist = np.argsort(point_dist)[:k] 
        #Labels of the K datapoints from above
        labels = y[dist]
        #Majority voting using mode()
        lab = mode(labels) 
        lab = lab.mode[0]
        op_labels.append(lab)
 
    return op_labels

# Feature Extraction

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa
vggmodel = hub.load('https://tfhub.dev/google/vggish/1')

def embedding_from_fn(fn):
    x, sr = librosa.load(fn,sr=None)
    x_16k = librosa.resample(x,sr,16000) #resample to 16KHz
    embedding = np.array(vggmodel(x_16k))
    return embedding

In [21]:
import numpy as np
import scipy.signal
import scipy.io.wavfile

fs,x = scipy.io.wavfile.read('./groove/drummer1/session1/3_funk_80_beat_4-4.wav')
audio=[]
for i in range(x.shape[0]):
    audio.append((x[i][0]+x[i][1])/2)
audio=np.array(audio)
audio = audio/max(audio)

<ipython-input-21-6b445b4c9d8c>:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs,x = scipy.io.wavfile.read('./groove/drummer1/session1/3_funk_80_beat_4-4.wav')
<ipython-input-21-6b445b4c9d8c>:8: RuntimeWarning: overflow encountered in short_scalars
  audio.append((x[i][0]+x[i][1])/2)


In [22]:
embedding = np.array(vggmodel(audio))

In [16]:
embedding.shape

(249, 128)

In [20]:
embedding.shape

(249, 128)

In [23]:
embedding.shape

(352, 128)

In [17]:
audio.shape

(3835150,)

In [ ]:
def block_audio(x,blockSize,hopSize,fs):    
    # allocate memory    
    numBlocks = math.ceil(x.size / hopSize)    
    xb = np.zeros([numBlocks, blockSize])    
    # compute time stamps    
    t = (np.arange(0, numBlocks) * hopSize) / fs   
    x = np.concatenate((x, np.zeros(blockSize)),axis=0)    
    for n in range(0, numBlocks):        
        i_start = n * hopSize        
        i_stop = np.min([x.size - 1, i_start + blockSize - 1])        
        xb[n][np.arange(0,blockSize)] = x[np.arange(i_start, i_stop + 1)]    
    return (xb,t)

In [ ]:
def agg_mfccs(x):
    mfccs = librosa.feature.mfcc(x,n_mfcc = 13)
    mean = [np.mean(feature) for feature in mfccs]
    std = [np.std(feature) for feature in mfccs]
    mfcc_delta = librosa.feature.delta(mfccs)
    delta_mean=[np.mean(feature) for feature in mfcc_delta]
    delta_std=[np.std(feature) for feature in mfcc_delta]
    return mean,std,delta_mean,delta_std